# RAG System from scratch

# 1. Load Documents

In [48]:
from pypdf import PdfReader
import re
import os 

file_path ="/Users/benjamin/src/Agentic-rag/data/Accident vasculaire cérébral (AVC) – Santé publique France.pdf"

reader = PdfReader(file_path)

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)


In [49]:
text = reader.pages
len(text)

4

In [50]:
text = reader.pages[2].extract_text()
text = text.replace('\xa0', ' ')  # espaces insécables
text = re.sub(r'\n+', '\n', text)  # supprimer les sauts de ligne multiples
text = re.sub(r'Page\s+\d+\s+sur\s+\d+', '', text)
text = re.sub(r'https?://\S+', '', text)  # supprimer les URLs
text = re.sub(r'\d{2}/\d{2}/\d{4}\s+\d{2}:\d{2}', '', text)
text = text.replace('\n','')
text

'Accident vasculaire cérébral (AVC) – Santé publique FranceLes chiffres-clés de l’AVCDes facteurs de risque à dépisterLa prévention des AVC repose principalement sur le dépistage et le traitement desfacteurs de risque vasculaires individuels (hypertension artérielle, tabagisme,hypercholestérolémie, diabète, troubles du rythme cardiaque, obésité, sédentarité).Des facteurs environnementaux comme la pollution atmosphérique ont également étéidenti;és comme facteurs de risque d’AVC. Renseignements administratifs des registres des AVCRegistre des accidents vasculaires cérébraux de DijonDirecteur : Professeur Maurice GIROUD, Médecin – PU-PHDirecteur scienti;que : Professeur Yannick BEJOT, Médecin - PU-PHRegistre Dijonnais des AVC, EA4184, UFR Sciences de Santé Dijon, Université deBourgogneHôpital François Mitterrand, 14 rue Paul Gaffarel, BP 7790821079 Dijon cedexRegistre des accidents vasculaires cérébraux de LilleResponsable scienti;que et administratifProfesseur Philippe AMOUYEL, Médecin –

## 2.Chunks the documents (we'll split the documents into pages for the exemple purpose)

In [51]:
def extract_pages(folder : str): 
    for filename in os.listdir(folder):
        
        if filename.endswith(".pdf"):
            document_path = os.path.join(folder, filename)
            reader = PdfReader(document_path)
            
        for i,page in enumerate(reader.pages):
            text = page.extract_text()
            text = text.replace('\xa0', ' ')  # espaces insécables
            text = re.sub(r'\n+', '\n', text)  # supprimer les sauts de ligne
            text = re.sub(r'Page\s+\d+\s+sur\s+\d+', '', text)
            text = re.sub(r'https?://\S+', '', text)  # supprimer les URL
            text = re.sub(r'\d{2}/\d{2}/\d{4}\s+\d{2}:\d{2}', '', text)
            text = text.replace('\n','')
            
            yield {
                "document": document_path,
                "page": i,
                "text": text
            }

In [52]:
for doc in extract_pages("/Users/benjamin/src/Agentic-rag/data"):
    print(doc["document"], doc["page"])
    print(doc["text"])
    print("\n")
    break  

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)


/Users/benjamin/src/Agentic-rag/data/Cancers de la peau | Santé publique France.pdf 0
Cancers de la peau | Santé publique FranceCancers de la peauAccueil Maladies et traumatismesCancersCancers de la peau : les maladiesLes cancers de la peau pourraient constituer le cancer le plus fréquent enFrance. Attribuables dans plus de 85% des cas à une exposition excessiveaux ultraviolets (UV) naturels ou artiAciels, ils peuvent être évités grâce àune exposition raisonnée. Chaque année, entre 141 200 et 243 500 cassont diagnostiqués en France.Mis à jour le 13 juillet 2023Nos missionsSurveiller l’évolution épidémiologique des cancers de la peau (mélanomes cutanéset autres cancers cutanés) Participer à l’expertise sur la surveillance épidémiologique et les politiques deprévention de l’exposition aux UVDécrire et suivre les comportements et les connaissances de la population surl’exposition aux UVInformer sur les dangers d’une exposition excessive aux UV et promouvoir lesprincipaux gestes de préven

In [54]:
docs = list(extract_pages("/Users/benjamin/src/Agentic-rag/data"))
docs[0]

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong p

{'document': '/Users/benjamin/src/Agentic-rag/data/Cancers de la peau | Santé publique France.pdf',
 'page': 0,
 'text': 'Cancers de la peau | Santé publique FranceCancers de la peauAccueil Maladies et traumatismesCancersCancers de la peau : les maladiesLes cancers de la peau pourraient constituer le cancer le plus fréquent enFrance. Attribuables dans plus de 85% des cas à une exposition excessiveaux ultraviolets (UV) naturels ou artiAciels, ils peuvent être évités grâce àune exposition raisonnée. Chaque année, entre 141 200 et 243 500 cassont diagnostiqués en France.Mis à jour le 13 juillet 2023Nos missionsSurveiller l’évolution épidémiologique des cancers de la peau (mélanomes cutanéset autres cancers cutanés) Participer à l’expertise sur la surveillance épidémiologique et les politiques deprévention de l’exposition aux UVDécrire et suivre les comportements et les connaissances de la population surl’exposition aux UVInformer sur les dangers d’une exposition excessive aux UV et promo

## 3. Indexing (embedding model)

In [55]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI()

In [56]:
sample = docs[0]["text"]

In [57]:
len(sample)

917

In [58]:
response = client.embeddings.create(
    input=sample,
    model="text-embedding-3-small"
)

response

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.00874314270913601, 0.04785945639014244, 0.021163657307624817, 0.023007705807685852, 0.03655930608510971, -0.013144196942448616, -0.011846930719912052, -0.015202668495476246, 0.0038757154252380133, 0.021999912336468697, -0.0033476960379630327, -0.0914304330945015, 0.027167532593011856, -0.02037028782069683, -0.009048696607351303, 0.04262750968337059, 0.044900406152009964, 0.0020678522996604443, -0.00442785769701004, -0.012533088214695454, 0.0313916876912117, -0.006448804400861263, 0.0012751532485708594, -0.04631560295820236, 0.02079913765192032, -0.046787336468696594, 0.015770891681313515, -0.04014018923044205, 0.0033905808813869953, 0.039389707148075104, 0.017743593081831932, -0.03147745504975319, 0.002219289308413863, 0.009445381350815296, -0.02253597229719162, -0.041898466646671295, 0.012886887416243553, 0.02040245197713375, -0.02071336656808853, 0.02324357070028782, 0.012543809600174427, -0.008850354701280594, 0.00369881559163332

In [60]:
embeddings = [item.embedding for item in response.data]
len(embeddings[0]) # 1536 dimensions for text-embedding-3-small

1536

In [67]:
def encode_documents(docs):
    for doc in docs:
        response = client.embeddings.create(
            input=doc["text"],
            model="text-embedding-3-small"
        )
        yield {
            "document": doc["document"],
            "page": doc["page"],
            "embedding": response.data[0].embedding
        }

In [ ]:
# Not working on Macos intell core

# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("BAAI/bge-m3")

In [69]:
import h5py
import numpy as np

embeddings = []
documents = []
pages = []

def write_embeddings_and_metadata(embeddings, documents, pages, filename = 'embeddings.hdf5') :
    utf8_type = h5py.string_dtype(encoding='utf-8')
    with h5py.File(filename,'w') as f : 
        f.create_dataset('embeddings', data=embeddings)
        f.create_dataset('documents', data = np.array(documents, dtype=utf8_type))
        f.create_dataset('pages', data = np.array(pages, dtype='i'))


def normalize(embedding) :
    norm = np.linalg.norm(embedding)
    return embedding / norm

for doc in encode_documents(docs):
    embedding = normalize(doc["embedding"])
    embeddings.append(embedding)
    documents.append(doc["document"])
    pages.append(doc["page"])
    
stacked_embeddings = np.stack(embeddings)
write_embeddings_and_metadata(stacked_embeddings, documents, pages)


In [70]:
def load_index(filename='embeddings.hdf5'):
    with h5py.File(filename, 'r') as f:
        embeddings = f['embeddings'][:]
        documents = f['documents'][:]
        pages = f['pages'][:]
    return embeddings, documents, pages


embeddings, documents, pages = load_index()

## 4. Calculate cosine similarity 

In [77]:
stacked_embeddings.shape

(175, 1536)

In [84]:
query = "Quelles sont les principales causes d'AVC ?"

query_response = client.embeddings.create(
            input=query,
            model="text-embedding-3-small"
        )

query_embedding = query_response.data[0].embedding
 
len(query_embedding)

1536

In [85]:
def calculate_cosine_similarity(embeddings, query) : 
    query = normalize(query)
    return embeddings @ query.T
   

In [ ]:
cosine_score = calculate_cosine_similarity(embeddings, query_embedding)
cosine_score

array([0.28211596, 0.26711238, 0.28245964, 0.33482823, 0.28899265,
       0.26248324, 0.28110766, 0.2648804 , 0.27275709, 0.34279899,
       0.3044261 , 0.30337429, 0.28442377, 0.30463484, 0.34492341,
       0.3322285 , 0.30053685, 0.30326842, 0.28078166, 0.32806868,
       0.34485317, 0.27241764, 0.27650953, 0.3076924 , 0.3071509 ,
       0.29018069, 0.26135866, 0.23942059, 0.26473671, 0.25233941,
       0.23060977, 0.17278841, 0.35165081, 0.31985537, 0.30489904,
       0.27105749, 0.27179348, 0.35422818, 0.41541255, 0.37806792,
       0.42491054, 0.50187318, 0.35762956, 0.36434342, 0.27654982,
       0.28292169, 0.33842113, 0.24483884, 0.23020214, 0.34730794,
       0.26015128, 0.2866849 , 0.29804592, 0.17049797, 0.23114088,
       0.26946007, 0.21742126, 0.27414897, 0.30112277, 0.22878569,
       0.33920963, 0.39816914, 0.29869326, 0.28141201, 0.30114969,
       0.26475397, 0.2111425 , 0.25208659, 0.56310242, 0.61688205,
       0.61448653, 0.49274104, 0.29036456, 0.2803142 , 0.26672

In [99]:
np.sort(cosine_score)[::-1]

array([0.61688205, 0.61448653, 0.56310242, 0.50187318, 0.49274104,
       0.44674218, 0.44229731, 0.42491054, 0.41541255, 0.40770718,
       0.40346428, 0.39816914, 0.37806792, 0.37013642, 0.36729902,
       0.36685835, 0.36646287, 0.36434342, 0.35762956, 0.35422818,
       0.35184011, 0.35165081, 0.34730794, 0.34492341, 0.34485317,
       0.34279899, 0.33920963, 0.33842113, 0.33482823, 0.3322285 ,
       0.32879134, 0.32806868, 0.32267048, 0.32249357, 0.31985537,
       0.31658776, 0.31622659, 0.31343935, 0.31288607, 0.31131675,
       0.3076924 , 0.3071509 , 0.30489904, 0.30463484, 0.3044261 ,
       0.3043446 , 0.3038962 , 0.30337429, 0.30326842, 0.30208313,
       0.30114969, 0.30112277, 0.30053685, 0.29869326, 0.29842646,
       0.29804592, 0.29774526, 0.29366064, 0.29364988, 0.2923841 ,
       0.29036456, 0.29018069, 0.28959507, 0.28899265, 0.28898799,
       0.28762121, 0.28752481, 0.2866849 , 0.28613649, 0.28594723,
       0.28442377, 0.28292169, 0.28245964, 0.28211596, 0.28141